In [1]:
from train_util import *

Current GPU device : 0


In [2]:
path_1 = "res_models/IMDB/bertbase/output_N_pro/checkpoint-2000"
path_2 = "res_models/IMDB/bertbase/output_N_pro/aa_early_stopping_5/checkpoint-3500"

tokenizet = load_hf('bertbase', load_model=False) #, path_to_model=path_1)


successfully loaded bert-base-uncased with BertTokenizer, BertForSequenceClassification


In [25]:
model = BertForSequenceClassification.from_pretrained(path_2, num_labels=2)

loading configuration file res_models/IMDB/bertbase/output_N_pro/aa_early_stopping_5/checkpoint-3500/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.12.5",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file res_models/IMDB/bertbase/output_N_pro/aa_early_stopping_5/checkpoint-3500/pytorch_model.bin
All model checkpoint w

In [4]:
type(model)

transformers.models.bert.modeling_bert.BertForSequenceClassification

In [11]:
filenames = glob(path_ + '*')
filenames.reverse()
print(filenames)
print('#################')
print(filenames[0])    
model_path = filenames[0]
print(model_path)
model = load_hf('bertbase', path_2) # DistilBertForSequenceClassification.from_pretrained(, num_labels=2) 


['res_models/IMDB/bertbase/output_N_pro/aa_early_stopping_5/checkpoint-3500']
#################
res_models/IMDB/bertbase/output_N_pro/aa_early_stopping_5/checkpoint-3500
res_models/IMDB/bertbase/output_N_pro/aa_early_stopping_5/checkpoint-3500
successfully loaded bert-base-uncased with BertTokenizer, BertForSequenceClassification


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [13]:
print(timestamp(True) + 'calculate accuracy with RESTRICTED test_set')
test_data_res = pd.read_pickle('res_data/IMDB_training/IMDB_N_pro_test')

print(timestamp(True)+ 'calculate accuracy with ALL test samples')
test_data_all = pd.read_pickle('res_data/IMDB_l_test')


test_data_res.label = pd.factorize(test_data_res.label)[0]
test_data_all.label = pd.factorize(test_data_all.label)[0]
if 'text' in test_data_res.columns[1]:
    test_data_res.rename(columns={test_data_res.columns[1]:'text'}, inplace=True)
else:
    print("ERROR: compute_metrics - wrong column renamed. This is not the text column")

if 'text' in test_data_all.columns[1]:
    test_data_all.rename(columns={test_data_all.columns[1]:'text'}, inplace=True)
else:
    print("ERROR: compute_metrics - wrong column renamed. This is not the text column")

01_10_1823calculate accuracy with RESTRICTED test_set
01_10_1823calculate accuracy with ALL test samples


In [18]:
tokenizer=tokenizet[0]

In [26]:
type(model)

transformers.models.bert.modeling_bert.BertForSequenceClassification

In [17]:
tokenizet

(PreTrainedTokenizer(name_or_path='bert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=False, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}),
 None)

In [27]:
# ----- Predict -----#
X_test = list(test_data_res["text"])
X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=512)

# Create torch dataset
test_dataset = Dataset(X_test_tokenized)

# Define test trainer
test_trainer = Trainer(model)

# Make prediction
raw_pred, _, _ = test_trainer.predict(test_dataset)

return_res = compute_metrics([raw_pred,list(test_data_res["label"])])

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running Prediction *****
  Num examples = 25000
  Batch size = 16


NameError: name 'test_data' is not defined

In [29]:
return_res = compute_metrics([raw_pred,list(test_data_res["label"])])
return_res

01_10_1843 : compute_metrics - accuracy: 0.9358; precision: 0.9387130546830957; recall: 0.93248; f1: 0.9355861460047357


{'accuracy': 0.9358,
 'precision': 0.9387130546830957,
 'recall': 0.93248,
 'f1': 0.9355861460047357}

In [30]:
# ----- Predict -----#
X_test = list(test_data_all["text"])
X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=512)

# Create torch dataset
test_dataset = Dataset(X_test_tokenized)

# Define test trainer
test_trainer = Trainer(model)

# Make prediction
raw_pred, _, _ = test_trainer.predict(test_dataset)

return_all = compute_metrics([raw_pred,list(test_data_all["label"])])

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running Prediction *****
  Num examples = 25000
  Batch size = 16


01_10_1849 : compute_metrics - accuracy: 0.93612; precision: 0.9399564199822452; recall: 0.93176; f1: 0.935840263549074


In [31]:
return_all

{'accuracy': 0.93612,
 'precision': 0.9399564199822452,
 'recall': 0.93176,
 'f1': 0.935840263549074}

In [ ]:
def calc_acc(spec, tokenizer, model_id, task="foo",  restricted_test_set = False):
    # ----- Load trained model -----#   
    path_ = "res_models/IMDB/bertbase/output_N_pro/aa_early_stopping_5/checkpoint-3500"
    print(path_)
    #filenames = next(walk(path_), (None, [], None))[1]  # [] if no file
    filenames = glob(path_ + '*')
    filenames.reverse()
    print(filenames)
    print('#################')
    print(filenames[0])    
    model_path = filenames[0]
    print(model_path)
    model = load_hf(model_id, path_to_model=model_path) # DistilBertForSequenceClassification.from_pretrained(, num_labels=2) 
    
    # ----- Load test data -----#
    if restricted_test_set: 
        print(timestamp(True) + 'calculate accuracy with RESTRICTED test_set')
        test_data = pd.read_pickle('res_data/{}_training/{}_{}_test'.format(task,task,spec))
    else:
        print(timestamp(True)+ 'calculate accuracy with ALL test samples')
        test_data = pd.read_pickle('res_data/' + task + '_l_test')
    test_data.label = pd.factorize(test_data.label)[0]
    if 'text' in test_data.columns[1]:
        test_data.rename(columns={test_data.columns[1]:'text'}, inplace=True)
    else:
        print("ERROR: compute_metrics - wrong column renamed. This is not the text column")
        # log.error(__name__ + ": compute_metrics - " + 'wrong column renamed. This is not the text column')          
        
    # ----- Predict -----#
    X_test = list(test_data["text"])
    X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=512)

    # Create torch dataset
    test_dataset = Dataset(X_test_tokenized)

    # Define test trainer
    test_trainer = Trainer(model)

    # Make prediction
    raw_pred, _, _ = test_trainer.predict(test_dataset)
    
    return(compute_metrics([raw_pred,list(test_data["label"])]))

In [9]:
path_ = "res_models/IMDB/bertbase/output_N_pro/aa_early_stopping_5/checkpoint-3500"
print(path_)


res_models/IMDB/bertbase/output_N_pro/aa_early_stopping_5/checkpoint-3500


In [7]:
acc_rest = calc_acc("N_pro", tokenizet, 'bertbase', task="IMDB", restricted_test_set = True)

file res_models/IMDB/bertbase/output_N_pro/aa_early_stopping_5/config.json not found


res_models/IMDB/bertbase/output_N_pro/
['res_models/IMDB/bertbase/output_N_pro/aa_early_stopping_5', 'res_models/IMDB/bertbase/output_N_pro/checkpoint-2000', 'res_models/IMDB/bertbase/output_N_pro/checkpoint-1500', 'res_models/IMDB/bertbase/output_N_pro/checkpoint-1000', 'res_models/IMDB/bertbase/output_N_pro/checkpoint-500', 'res_models/IMDB/bertbase/output_N_pro/runs']
#################
res_models/IMDB/bertbase/output_N_pro/aa_early_stopping_5
res_models/IMDB/bertbase/output_N_pro/aa_early_stopping_5
successfully loaded bert-base-uncased with BertTokenizer, BertForSequenceClassification
res_models/IMDB/bertbase/output_N_pro/aa_early_stopping_5


OSError: Can't load config for 'res_models/IMDB/bertbase/output_N_pro/aa_early_stopping_5'. Make sure that:

- 'res_models/IMDB/bertbase/output_N_pro/aa_early_stopping_5' is a correct model identifier listed on 'https://huggingface.co/models'
  (make sure 'res_models/IMDB/bertbase/output_N_pro/aa_early_stopping_5' is not a path to a local directory with something else, in that case)

- or 'res_models/IMDB/bertbase/output_N_pro/aa_early_stopping_5' is the correct path to a directory containing a config.json file

